# Twitter(X)API取得コード

#### [使い始め時の使い方 for beginner]
1. カーネル ".venv(Python 3.12.9) **All_collect_tweet/Forcoltweet/.venv/bin/python**"を選んでください.
2. 【毎度いじるパラメータ】で、"取得する期間"と、"取得するKW"を設定してください.
3. 期間を半日にして、一度取得実験をし、中身が揃っていることを確認してください.
4. 大型取得を開始してください.
* 取得は、今後のデータ処理のことも考えると、期間設定を1ヶ月ごとに取得してください.

#### [参考]
* KWが「日経平均」の場合、1ヶ月で約3万-5万ツイート.
    * 2025/6/25現在、Pro契約最大100万ツイートまで取得可能. したがって、おおよそ20ヶ月程度取得可能かと思われます.
* 100万ツイート取得するのに、本気で画面と向き合えば、10時間で取得完了します.
    * 傾向として、取得よりcsv化の方が、処理時間が長い印象です.
* 429のエラーが出ているときは、リクエスト制限かかっている可能性があります. 5分程度空けてから実行すると解決するかもしれないです.

#### [コード修正をする際の注意点]
* APIに取得依頼→csv化してデータ保存の流れです.
    * したがって、query_params_base (APIに注文する内容) にあっても row(csvに出力させるときの列) に入れなければ CSV に出力されない です.
    * 逆に row 側で入れていても APIで取得してないものは空になります（例: profile_banner_url）.
* 本コードは、query_params_baseのほぼ全てを入れているので、row の部分に要素を追加すれば、取得情報を増やせられます. 詳しくはChatGPT等に聞くと正確に返ってくるかと思います.
    * context_annotation に関しては、いれると、おそらく動かなくなります. もし仮に使いたい人いれば、MAX_RESULT辺りを小さく(200くらい)してみてください.
* 仮想環境に入っているライブラリ一覧を確認したい場合は、同じディレクトリにある pyproject.tomlファイル を参照してください.
    * ターミナルを開き、次のコマンド *uv add [ライブラリ名]* で、pip install と同様のインストールができます.

In [ ]:
import time
import requests
from dotenv import load_dotenv
import os
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm
tqdm.pandas()

# =============================
# パラメータ設定
# =============================

# 【毎度いじるパラメータ】~~~~~~~~~~~~
START_TIME = datetime(2023, 11, 1, 0, 0, 0) #取得開始日JST（YYYY-MM-DD HH:MM:SS）
END_TIME = datetime(2023, 11, 1, 23, 59, 59) #取得終了日JST（YYYY-MM-DD HH:MM:SS）
QUERY = "日経平均 -is:retweet" #抽出キーワード
'''
<QUERY = "日経平均 -is:retweet"に関して>
日経平均：「日経平均」という文字列を含むツイートを検索
-is:retweet:かつ、リツイート（RT）は除外する
'''
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

MAX_RESULTS = 500 #安田さん500|1リクエストで取得するツイート数の最大値//10以上でないと動かない.
MAX_PAGES = 100000 #安田さん100000|最大で何回リクエストを送るかの上限|参照元の安田さんのコードでは、response_count = 10000として定義していた.
'''
● MAX_RESULTS = 2、MAX_PAGES = 3 の場合
ページ1 → 2件取得  
ページ2 → 2件取得  
ページ3 → 2件取得  
== 合計6件取得
--------------------------------------------
● MAX_RESULTS = 2、MAX_PAGES = 1 の場合
ページ1 → 2件取得   
== 合計2件取得
'''

#.envを使うなら(動作保証なし)
load_dotenv("/home/sakulab/workspace/All_collect_tweet/Forcoltweet/.env")
BEARER_TOKEN = os.getenv('BEARER_TOKEN')

#BEARER_TOKEN = 触らない|ベタ打ち
OUTPUT_DIRECTORY = "/home/sakulab/workspace/All_collect_tweet/collected_tweet" #csv出力先のファイルパス
if not BEARER_TOKEN:
    print("Warning: BEARER_TOKEN is not set in the .env file")
else:
    print("BEARER_TOKENは正常に取得されました")

# =============================
# 時間変換(JST→UTC)
# =============================
JST = timedelta(hours=9) # 日本標準時 (JST) は UTC +9時間なので、その9時間を引いてUTCに変換するため.
start_time_utc = (START_TIME - JST).isoformat() + "Z" # 開始日時をUTCに変換し、ISOフォーマット + Z（ゼロ時区）で文字列化
end_time_utc = (END_TIME - JST).isoformat() + "Z" # 終了日時を同様にUTC変換・フォーマット

# =============================
# API設定
# =============================
url = "https://api.X.com/2/tweets/search/all" #proプランでの収集用(本家)
#url = "https://api.X.com/2/tweets/search/recent"#freeプラン収集用(最大10件程度)

headers = {"Authorization": f"Bearer {BEARER_TOKEN}"} # リクエストヘッダーに Bearer トークンをセット（認証用）

#取得要素の設定
query_params_base = {
    "query": QUERY, # 検索クエリ（キーワード＋条件、例：日経平均 -is:retweet）
    "start_time": start_time_utc, # 検索開始日時（UTC ISO形式）
    "end_time": end_time_utc, # 検索終了日時（UTC ISO形式）
    "max_results": MAX_RESULTS, # 1リクエストあたりの最大取得件数（10～500の範囲）
    "tweet.fields": ",".join([ # ツイートから取得したいフィールド一覧
        "attachments", "author_id", "conversation_id",#"context_annotations",
        "created_at", "entities", "geo", "id", "in_reply_to_user_id", "lang",
        "public_metrics", "possibly_sensitive", "referenced_tweets",
        "reply_settings", "source", "text", "withheld"
    ]),
    "expansions": "author_id,attachments.media_keys,referenced_tweets.id",  # 結果に含める追加データ（例：ユーザー情報・メディア情報）
    "user.fields": ",".join([ # ユーザー情報として取得するフィールド
        "created_at", "description", "entities", "id", "location", "name",
        "pinned_tweet_id", "profile_image_url", "protected", "public_metrics",
        "url", "username", "verified", "withheld"
    ]),
    "media.fields": "media_key,type,url,preview_image_url,public_metrics", # メディア情報で取得するフィールド
    "place.fields": "full_name,id,country,country_code,geo,name,place_type" # 場所情報で取得するフィールド
} 

# =============================
# データ収集関数
# =============================
def collect_tweet(url, headers, base_params):
    result = [] # すべてのツイートデータを格納するリスト
    tweet_count = 0 #実行中のprint用
    all_users = {} # ユーザー情報を author_id をキーに格納する辞書
    all_media = {} # メディア情報を media_key をキーに格納する辞書
    all_places = {} # 場所情報を place_id をキーに格納する辞書
    next_token = None # ページング用のトークン（最初はNone）
    count = 0 # リクエスト回数のカウンタ

    while count < MAX_PAGES: # 最大リクエスト回数に達するまでループ
        time.sleep(1) # APIレート制限対策でリクエスト間に1秒の遅延
        params = base_params.copy()# base_params（基本のリクエストパラメータ）をコピーして、新しいparamsを作成
        if next_token:
            params["next_token"] = next_token # 続きを取得する場合は next_token を指定

        response = requests.get(url, headers=headers, params=params) # APIリクエスト送信

        if response.status_code != 200:
            raise Exception(f"Request failed: {response.status_code}, {response.text}") # エラー時は例外を発生させ、レスポンス内容を表示|例)status-code=404→ページ見つからない

        body = response.json() # JSONレスポンスを辞書化
        result.extend(body.get("data", [])) # ツイートデータを結果リストに追加

        # 累積件数とレート制限の残りを出力
        rate_limit = response.headers.get('x-rate-limit-remaining', 'N/A')
        tweet_count += len(body.get("data", []))

        if body.get("data"):
            created_at_jst = datetime.fromisoformat(body["data"][0]["created_at"].replace("Z", "+00:00")) + JST
            print(f"Rate limit remaining: {rate_limit} | 目安の日付: {created_at_jst} | 累計Tweet_count: {tweet_count}")
        else:
            print(f"Rate limit remaining: {rate_limit} | データなし | 累計Tweet_count: {tweet_count}")

        # 各種インクルードデータ（ユーザー・メディア・場所情報）を辞書に格納
        for user in body.get("includes", {}).get("users", []):
            all_users[user["id"]] = user
        for media in body.get("includes", {}).get("media", []):
            all_media[media["media_key"]] = media
        for place in body.get("includes", {}).get("places", []):
            all_places[place["id"]] = place

        count += 1 # リクエスト回数をインクリメント
        next_token = body.get("meta", {}).get("next_token")
        if not next_token:
            print("No more pages to fetch.")
            break # 次トークンが無ければこれ以上のデータはないので終了

    return result, all_users, all_media, all_places # 収集した全データを返す

# =============================
# データ収集実行
# =============================
#tweets, users_dict, media_dict, places_dict = collect_tweet(url, headers, query_params_base)
tweets, users_dict, media_dict, places_dict = collect_tweet(
    url,                 # APIエンドポイントのURL (search/all)
    headers,             # リクエストの認証ヘッダー (Bearerトークン)
    query_params_base    # 検索条件・取得フィールドなどを含むパラメータ
)
'''
# collect_tweet 関数を呼び出して、指定の検索条件でデータ収集を実行
# 戻り値は以下の4つ:
# tweets: ツイートデータ（リスト形式、ツイート1件ずつが辞書）
# users_dict: 投稿者ユーザー情報（ユーザーIDをキー、値がユーザー情報の辞書）
# media_dict: 添付メディア情報（media_key をキー、値がメディア情報の辞書）
# places_dict: 場所情報（place_id をキー、値が場所情報の辞書）
'''

# =============================
# csv化,データを1行にまとめる
# =============================
rows = []# rows はツイート＋ユーザー＋メディア＋場所情報を統合したデータを1行ずつ格納するためのリスト.最終的に pandas DataFrame に変換するために使用.

for tweet in tqdm(tweets, desc="全情報統合中"):
    created_at_jst = (datetime.fromisoformat(tweet["created_at"].replace("Z", "+00:00")) + JST) if tweet.get("created_at") else ""
    # ツイートのcreated_at（UTC:Z=ゼロ時区）をdatetime型に変換し、日本時間 (JST) に変換
    # created_at が無ければ空文字

    user = users_dict.get(tweet.get("author_id", ""), {}) # ツイートの author_id から対応するユーザー情報を取得 / 存在しない場合は空の辞書
    user_metrics = user.get("public_metrics", {}) # ユーザーの public_metrics を取得（フォロワー数やツイート数など）/ 無ければ空の辞書
    metrics = tweet.get("public_metrics", {}) # ツイートの public_metrics を取得（リツイート数、いいね数など）| 無ければ空の辞書

    # メディア情報（複数ある場合は最初のもののみ例として取り出し）
    media_info = {} # メディア情報（画像・動画など）を格納する辞書を初期化
    media_keys = tweet.get("attachments", {}).get("media_keys", []) # ツイートにメディアが添付されている場合、その media_keys のリストを取得
    if media_keys:
        media_info = media_dict.get(media_keys[0], {}) # media_keys が存在する場合、最初の media_key に対応するメディア情報を取得

    # 場所情報（geo.place_id があれば）
    place_info = {} # 場所情報（geo.place_id があればそのIDに対応する場所情報を取得）を格納する辞書を初期化
    geo = tweet.get("geo", {}) # geo 情報を取得（ツイート投稿時に位置情報が付いていれば入っている）
    if geo.get("place_id"):
        place_info = places_dict.get(geo["place_id"], {}) # geo 内に place_id が存在する場合、そのIDから場所情報を取得

    row = {
        # ツイート情報
        "text": tweet.get("text", ""), # ツイート本文
        #"text": tweet.get("text", "").replace("\n", " "), # ツイート本文（改行除去）
        "created_at": tweet["created_at"], # ツイート投稿日時（UTC / 英国時間）
        "created_at_jst": created_at_jst.strftime("%Y-%m-%d %H:%M:%S") if created_at_jst else "", # 日本時間の投稿日時（YYYY-MM-DD HH:MM:SS）
        "tweet_id": tweet.get("id", ""), # ツイート固有のID
        "author_id": tweet.get("author_id", ""), # ツイート投稿者のユーザーID
        "lang": tweet.get("lang", ""), # ツイート言語（例: ja:日本語, en:英語）
        "source": tweet.get("source", ""), # ツイート投稿元アプリ・クライアント名
        "retweet_count": metrics.get("retweet_count", 0), # リツイート数
        "reply_count": metrics.get("reply_count", 0), # リプライ数
        "like_count": metrics.get("like_count", 0), # いいね数
        "quote_count": metrics.get("quote_count", 0), # 引用ツイート数
        "possibly_sensitive": tweet.get("possibly_sensitive", False), # センシティブフラグ（Trueならセンシティブ）

        # ユーザー情報
        "user_id": user.get("id", ""), # ユーザーID（author_idと同じはず）
        "username": user.get("username", ""), # ユーザーのスクリーンネーム（@名）
        "name": user.get("name", ""), # ユーザーの表示名
        "verified": user.get("verified", False), # 認証済みアカウントかどうか（True/False）
        "followers_count": user_metrics.get("followers_count", 0), # フォロワー数
        "following_count": user_metrics.get("following_count", 0), # フォロー数
        "tweet_count": user_metrics.get("tweet_count", 0), # トータルツイート数
        "listed_count": user_metrics.get("listed_count", 0), # リスト登録数
        "user_location": user.get("location", ""), # ユーザーの登録場所情報
        "profile_image_url": user.get("profile_image_url", ""), # プロフィール画像URL
        #"description": user.get("description", "").replace("\n", " "), # プロフィール文
        "description": user.get("description", "").replace("\n", " "), # プロフィール文（改行除去）
        "created_at_user": user.get("created_at", ""), # アカウント作成日時（UTC）
        "url": user.get("url", ""), # ユーザのWebサイトURL
        "pinned_tweet_id": user.get("pinned_tweet_id", ""), # ピン留めツイートのID
        "profile_banner_url": user.get("profile_banner_url", ""), # プロフィールバナー画像URL

        # メディア情報（あれば）
        "media_type": media_info.get("type", ""), # メディアタイプ（photo, video, animated_gif など）
        "media_url": media_info.get("url", ""), # メディアのURL（画像・動画）
        "media_preview_image_url": media_info.get("preview_image_url", ""), # 動画のプレビュー画像URL

        # 場所情報|ツイートの投稿時に紐づけられた場所情報（あれば.ほとんど空）
        "place_name": place_info.get("name", ""), # 場所の名称（市区町村など）
        "place_country": place_info.get("country", ""), # 国名
        "place_type": place_info.get("place_type", "") # 場所タイプ（city, admin, country など）
    }
    rows.append(row)# インデント内(for文内に入ってることを確認してください)

# =============================
# CSVに出力
# =============================
df = pd.DataFrame(rows) #全てをDataframe化
df = df.sort_values(by="created_at")  # created_at の昇順にソート
start_str = START_TIME.strftime("%Y%m%d_%H%M%S") #ファイル名にする開始日時(JST)
end_str = END_TIME.strftime("%Y%m%d_%H%M%S") #ファイル名にする終了日時(JST)
output_path = f"{OUTPUT_DIRECTORY}/tweets_allinfo_{start_str}_to_{end_str}.csv" #ファイルパスの指定/ファイル名の指定
df.to_csv(output_path, index=False, encoding="utf-8-sig") #csv化

print(f"保存しました: {output_path}")

Rate limit remaining: 299 | 目安の日付: 2023-11-01 23:58:43+00:00 | 累計Tweet_count: 379
Rate limit remaining: 298 | 目安の日付: 2023-11-01 19:11:22+00:00 | 累計Tweet_count: 765
Rate limit remaining: 297 | 目安の日付: 2023-11-01 15:59:09+00:00 | 累計Tweet_count: 1101
Rate limit remaining: 296 | 目安の日付: 2023-11-01 14:52:24+00:00 | 累計Tweet_count: 1489
Rate limit remaining: 295 | 目安の日付: 2023-11-01 11:45:32+00:00 | 累計Tweet_count: 1865
Rate limit remaining: 294 | 目安の日付: 2023-11-01 10:05:26+00:00 | 累計Tweet_count: 2224
Rate limit remaining: 293 | 目安の日付: 2023-11-01 08:30:02+00:00 | 累計Tweet_count: 2569


全情報統合中: 100%|██████████| 2569/2569 [00:00<00:00, 137340.25it/s]

保存しました: /home/sakulab/workspace/All_collect_tweet/collected_tweet/tweets_allinfo_20231101_000000_to_20231101_235959.csv


# 分割版

In [ ]:
import time
import requests
from dotenv import load_dotenv
import os
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm
tqdm.pandas()

# =============================
# パラメータ設定
# =============================

# 【毎度いじるパラメータ】~~~~~~~~~~~~
START_TIME = datetime(2023, 11, 1, 0, 0, 0) #取得開始日JST（YYYY-MM-DD HH:MM:SS）
END_TIME = datetime(2023, 11, 1, 23, 59, 59) #取得終了日JST（YYYY-MM-DD HH:MM:SS）
QUERY = "日経平均 -is:retweet" #抽出キーワード
'''
<QUERY = "日経平均 -is:retweet"に関して>
日経平均：「日経平均」という文字列を含むツイートを検索
-is:retweet:かつ、リツイート（RT）は除外する
'''
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

MAX_RESULTS = 500 #安田さん500|1リクエストで取得するツイート数の最大値//10以上でないと動かない.
MAX_PAGES = 100000 #安田さん100000|最大で何回リクエストを送るかの上限|参照元の安田さんのコードでは、response_count = 10000として定義していた.
'''
● MAX_RESULTS = 2、MAX_PAGES = 3 の場合
ページ1 → 2件取得  
ページ2 → 2件取得  
ページ3 → 2件取得  
== 合計6件取得
--------------------------------------------
● MAX_RESULTS = 2、MAX_PAGES = 1 の場合
ページ1 → 2件取得   
== 合計2件取得
'''

#.envを使うなら(動作保証なし)
load_dotenv("/home/sakulab/workspace/All_collect_tweet/Forcoltweet/.env")
BEARER_TOKEN = os.getenv('BEARER_TOKEN')

#BEARER_TOKEN = 'A'#▲触らない|ベタ打ち
OUTPUT_DIRECTORY = "/home/sakulab/workspace/All_collect_tweet/collected_tweet" #csv出力先のファイルパス
if not BEARER_TOKEN:
    print("Warning: BEARER_TOKEN is not set in the .env file")
else:
    print("BEARER_TOKENは正常に取得されました")

# =============================
# 時間変換(JST→UTC)
# =============================
JST = timedelta(hours=9) # 日本標準時 (JST) は UTC +9時間なので、その9時間を引いてUTCに変換するため.
start_time_utc = (START_TIME - JST).isoformat() + "Z" # 開始日時をUTCに変換し、ISOフォーマット + Z（ゼロ時区）で文字列化
end_time_utc = (END_TIME - JST).isoformat() + "Z" # 終了日時を同様にUTC変換・フォーマット

# =============================
# API設定
# =============================
url = "https://api.X.com/2/tweets/search/all" #proプランでの収集用(本家)
#url = "https://api.X.com/2/tweets/search/recent"#freeプラン収集用(最大10件程度)

headers = {"Authorization": f"Bearer {BEARER_TOKEN}"} # リクエストヘッダーに Bearer トークンをセット（認証用）

#取得要素の設定
query_params_base = {
    "query": QUERY, # 検索クエリ（キーワード＋条件、例：日経平均 -is:retweet）
    "start_time": start_time_utc, # 検索開始日時（UTC ISO形式）
    "end_time": end_time_utc, # 検索終了日時（UTC ISO形式）
    "max_results": MAX_RESULTS, # 1リクエストあたりの最大取得件数（10～500の範囲）
    "tweet.fields": ",".join([ # ツイートから取得したいフィールド一覧
        "attachments", "author_id", "conversation_id",#"context_annotations",
        "created_at", "entities", "geo", "id", "in_reply_to_user_id", "lang",
        "public_metrics", "possibly_sensitive", "referenced_tweets",
        "reply_settings", "source", "text", "withheld"
    ]),
    "expansions": "author_id,attachments.media_keys,referenced_tweets.id",  # 結果に含める追加データ（例：ユーザー情報・メディア情報）
    "user.fields": ",".join([ # ユーザー情報として取得するフィールド
        "created_at", "description", "entities", "id", "location", "name",
        "pinned_tweet_id", "profile_image_url", "protected", "public_metrics",
        "url", "username", "verified", "withheld"
    ]),
    "media.fields": "media_key,type,url,preview_image_url,public_metrics", # メディア情報で取得するフィールド
    "place.fields": "full_name,id,country,country_code,geo,name,place_type" # 場所情報で取得するフィールド
} 

# =============================
# データ収集関数
# =============================
def collect_tweet(url, headers, base_params):
    result = [] # すべてのツイートデータを格納するリスト
    tweet_count = 0 #実行中のprint用
    all_users = {} # ユーザー情報を author_id をキーに格納する辞書
    all_media = {} # メディア情報を media_key をキーに格納する辞書
    all_places = {} # 場所情報を place_id をキーに格納する辞書
    next_token = None # ページング用のトークン（最初はNone）
    count = 0 # リクエスト回数のカウンタ

    while count < MAX_PAGES: # 最大リクエスト回数に達するまでループ
        time.sleep(1) # APIレート制限対策でリクエスト間に1秒の遅延
        params = base_params.copy()# base_params（基本のリクエストパラメータ）をコピーして、新しいparamsを作成
        if next_token:
            params["next_token"] = next_token # 続きを取得する場合は next_token を指定

        response = requests.get(url, headers=headers, params=params) # APIリクエスト送信

        if response.status_code != 200:
            raise Exception(f"Request failed: {response.status_code}, {response.text}") # エラー時は例外を発生させ、レスポンス内容を表示|例)status-code=404→ページ見つからない

        body = response.json() # JSONレスポンスを辞書化
        result.extend(body.get("data", [])) # ツイートデータを結果リストに追加

        # 累積件数とレート制限の残りを出力
        rate_limit = response.headers.get('x-rate-limit-remaining', 'N/A')
        tweet_count += len(body.get("data", []))

        if body.get("data"):
            created_at_jst = datetime.fromisoformat(body["data"][0]["created_at"].replace("Z", "+00:00")) + JST
            print(f"Rate limit remaining: {rate_limit} | 目安の日付: {created_at_jst} | 累計Tweet_count: {tweet_count}")
        else:
            print(f"Rate limit remaining: {rate_limit} | データなし | 累計Tweet_count: {tweet_count}")

        # 各種インクルードデータ（ユーザー・メディア・場所情報）を辞書に格納
        for user in body.get("includes", {}).get("users", []):
            all_users[user["id"]] = user
        for media in body.get("includes", {}).get("media", []):
            all_media[media["media_key"]] = media
        for place in body.get("includes", {}).get("places", []):
            all_places[place["id"]] = place

        count += 1 # リクエスト回数をインクリメント
        next_token = body.get("meta", {}).get("next_token")
        if not next_token:
            print("No more pages to fetch.")
            break # 次トークンが無ければこれ以上のデータはないので終了

    return result, all_users, all_media, all_places # 収集した全データを返す

# =============================
# データ収集実行
# =============================
#tweets, users_dict, media_dict, places_dict = collect_tweet(url, headers, query_params_base)
tweets, users_dict, media_dict, places_dict = collect_tweet(
    url,                 # APIエンドポイントのURL (search/all)
    headers,             # リクエストの認証ヘッダー (Bearerトークン)
    query_params_base    # 検索条件・取得フィールドなどを含むパラメータ
)
'''
# collect_tweet 関数を呼び出して、指定の検索条件でデータ収集を実行
# 戻り値は以下の4つ:
# tweets: ツイートデータ（リスト形式、ツイート1件ずつが辞書）
# users_dict: 投稿者ユーザー情報（ユーザーIDをキー、値がユーザー情報の辞書）
# media_dict: 添付メディア情報（media_key をキー、値がメディア情報の辞書）
# places_dict: 場所情報（place_id をキー、値が場所情報の辞書）
'''

In [ ]:
# =============================
# csv化,データを1行にまとめる
# =============================
rows = []# rows はツイート＋ユーザー＋メディア＋場所情報を統合したデータを1行ずつ格納するためのリスト.最終的に pandas DataFrame に変換するために使用.

for tweet in tqdm(tweets, desc="全情報統合中"):
    created_at_jst = (datetime.fromisoformat(tweet["created_at"].replace("Z", "+00:00")) + JST) if tweet.get("created_at") else ""
    # ツイートのcreated_at（UTC:Z=ゼロ時区）をdatetime型に変換し、日本時間 (JST) に変換
    # created_at が無ければ空文字

    user = users_dict.get(tweet.get("author_id", ""), {}) # ツイートの author_id から対応するユーザー情報を取得 / 存在しない場合は空の辞書
    user_metrics = user.get("public_metrics", {}) # ユーザーの public_metrics を取得（フォロワー数やツイート数など）/ 無ければ空の辞書
    metrics = tweet.get("public_metrics", {}) # ツイートの public_metrics を取得（リツイート数、いいね数など）| 無ければ空の辞書

    # メディア情報（複数ある場合は最初のもののみ例として取り出し）
    media_info = {} # メディア情報（画像・動画など）を格納する辞書を初期化
    media_keys = tweet.get("attachments", {}).get("media_keys", []) # ツイートにメディアが添付されている場合、その media_keys のリストを取得
    if media_keys:
        media_info = media_dict.get(media_keys[0], {}) # media_keys が存在する場合、最初の media_key に対応するメディア情報を取得

    # 場所情報（geo.place_id があれば）
    place_info = {} # 場所情報（geo.place_id があればそのIDに対応する場所情報を取得）を格納する辞書を初期化
    geo = tweet.get("geo", {}) # geo 情報を取得（ツイート投稿時に位置情報が付いていれば入っている）
    if geo.get("place_id"):
        place_info = places_dict.get(geo["place_id"], {}) # geo 内に place_id が存在する場合、そのIDから場所情報を取得

    row = {
        # ツイート情報
        "text": tweet.get("text", ""), # ツイート本文
        #"text": tweet.get("text", "").replace("\n", " "), # ツイート本文（改行除去）
        "created_at": tweet["created_at"], # ツイート投稿日時（UTC / 英国時間）
        "created_at_jst": created_at_jst.strftime("%Y-%m-%d %H:%M:%S") if created_at_jst else "", # 日本時間の投稿日時（YYYY-MM-DD HH:MM:SS）
        "tweet_id": tweet.get("id", ""), # ツイート固有のID
        "author_id": tweet.get("author_id", ""), # ツイート投稿者のユーザーID
        "lang": tweet.get("lang", ""), # ツイート言語（例: ja:日本語, en:英語）
        "source": tweet.get("source", ""), # ツイート投稿元アプリ・クライアント名
        "retweet_count": metrics.get("retweet_count", 0), # リツイート数
        "reply_count": metrics.get("reply_count", 0), # リプライ数
        "like_count": metrics.get("like_count", 0), # いいね数
        "quote_count": metrics.get("quote_count", 0), # 引用ツイート数
        "possibly_sensitive": tweet.get("possibly_sensitive", False), # センシティブフラグ（Trueならセンシティブ）

        # ユーザー情報
        "user_id": user.get("id", ""), # ユーザーID（author_idと同じはず）
        "username": user.get("username", ""), # ユーザーのスクリーンネーム（@名）
        "name": user.get("name", ""), # ユーザーの表示名
        "verified": user.get("verified", False), # 認証済みアカウントかどうか（True/False）
        "followers_count": user_metrics.get("followers_count", 0), # フォロワー数
        "following_count": user_metrics.get("following_count", 0), # フォロー数
        "tweet_count": user_metrics.get("tweet_count", 0), # トータルツイート数
        "listed_count": user_metrics.get("listed_count", 0), # リスト登録数
        "user_location": user.get("location", ""), # ユーザーの登録場所情報
        "profile_image_url": user.get("profile_image_url", ""), # プロフィール画像URL
        #"description": user.get("description", "").replace("\n", " "), # プロフィール文
        "description": user.get("description", "").replace("\n", " "), # プロフィール文（改行除去）
        "created_at_user": user.get("created_at", ""), # アカウント作成日時（UTC）
        "url": user.get("url", ""), # ユーザのWebサイトURL
        "pinned_tweet_id": user.get("pinned_tweet_id", ""), # ピン留めツイートのID
        "profile_banner_url": user.get("profile_banner_url", ""), # プロフィールバナー画像URL

        # メディア情報（あれば）
        "media_type": media_info.get("type", ""), # メディアタイプ（photo, video, animated_gif など）
        "media_url": media_info.get("url", ""), # メディアのURL（画像・動画）
        "media_preview_image_url": media_info.get("preview_image_url", ""), # 動画のプレビュー画像URL

        # 場所情報|ツイートの投稿時に紐づけられた場所情報（あれば.ほとんど空）
        "place_name": place_info.get("name", ""), # 場所の名称（市区町村など）
        "place_country": place_info.get("country", ""), # 国名
        "place_type": place_info.get("place_type", "") # 場所タイプ（city, admin, country など）
    }
    rows.append(row)# インデント内(for文内に入ってることを確認してください)

# =============================
# CSVに出力
# =============================
df = pd.DataFrame(rows) #全てをDataframe化
df = df.sort_values(by="created_at")  # created_at の昇順にソート
start_str = START_TIME.strftime("%Y%m%d_%H%M%S") #ファイル名にする開始日時(JST)
end_str = END_TIME.strftime("%Y%m%d_%H%M%S") #ファイル名にする終了日時(JST)
output_path = f"{OUTPUT_DIRECTORY}/tweets_allinfo_{start_str}_to_{end_str}.csv" #ファイルパスの指定/ファイル名の指定
df.to_csv(output_path, index=False, encoding="utf-8-sig") #csv化

print(f"保存しました: {output_path}")